In [1]:
from tensorflow import keras
import pickle
import networkx as nx
import numpy as np
from networkx.utils import reverse_cuthill_mckee_ordering
import pandas as pd
from PIL import Image

In [2]:
# NUMBER_NODES = 10
# NUMBER_NODES = 9
NUMBER_NODES = 7

In [3]:
model = keras.models.load_model('./CNN/CNN_dir_adj2_model')

In [4]:
def load_test_data():
    NUMBER_NODES=7
    df = pd.read_csv(f'./CNN/test_adj2_graphs_{NUMBER_NODES}/teste_data2_{NUMBER_NODES}.csv')
    print(df.shape)

    featuresNumber = ((NUMBER_NODES * NUMBER_NODES - NUMBER_NODES) // 2 )
    features = df.iloc[ : , : featuresNumber ].values
    labels = df.iloc[ : , featuresNumber + 1 : ].values # skip bandwidth value
    return features, labels

In [5]:
test_features, test_labels = load_test_data()
print(test_features.shape)
print(test_labels.shape)

(104, 29)
(104, 21)
(104, 7)


In [6]:
def get_bandwidth_nn_output(Graph, nodelist):
    Graph = nx.Graph(Graph)
    if not Graph.edges:
        return 0
    L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    x, y = np.nonzero(L)
    return (x-y).max()

def processDataToAdjImage(graphInput):
    adj = getGraph(graphInput)
    w, h = NUMBER_NODES, NUMBER_NODES
    data = np.zeros((h, w, 3), dtype=np.uint8)
    for i in range(len(adj)):
        for j in range(len(adj)):
            if adj[i, j] == 1:
                data[i, j] = np.array([255.0, 255.0, 255.0])
    # data /= 255.0
    img = Image.fromarray(data, 'RGB')
    resized = img.resize((32, 32), Image.NEAREST)
    image_input_np = np.array(resized)
    return image_input_np

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj

def getArray(strArray):
    arr = strArray.split('.')
    first = strArray[1]
    arr = [first, *arr[1 : -1]] # skip '[' and ']'
    arr = list(map(lambda x: int(x.strip()), arr))
    return arr

In [7]:
def getCuthillMckeeBand(Graph):
    rcm = list(reverse_cuthill_mckee_ordering(Graph))
    L = nx.laplacian_matrix(Graph, nodelist=rcm)
    x, y = np.nonzero(L)
    return (x-y).max()

def get_bandwidth(Graph, nodelist):
    L = nx.laplacian_matrix(Graph, nodelist=nodelist)
    x, y = np.nonzero(L)
    if not Graph.edges:
        return 0
    return (x-y).max()

def getGraph(upperTriangleAdjMatrix):
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    dense_adj = np.zeros((NUMBER_NODES, NUMBER_NODES))
    k = 0
    for i in range(NUMBER_NODES):
        for j in range(NUMBER_NODES):
            if i == j:
                continue
            elif i < j:
                dense_adj[i][j] = upperTriangleAdjMatrix[k]
                k += 1
            else:
                dense_adj[i][j] = dense_adj[j][i]
    return dense_adj


In [8]:
with open('./CNN/class_names.pickle', 'rb') as f:
    class_names = pickle.load(f)

In [9]:
len(class_names)

452

In [10]:
df = pd.DataFrame([])

sum_true = 0
sum_pred = 0
sum_rcm = 0

for x, true in zip(test_features, test_labels):
    get_graph_adj_image_arr = processDataToAdjImage(x)
    pred = model.predict(np.array([get_graph_adj_image_arr]))

    str_pred_nodelist = class_names[np.argmax(pred)]
    true_nodelist = list(map(int, true))

    arr_pred_nodelist = getArray(str_pred_nodelist)
    arr_true_nodelist = true_nodelist
    adj_graph = getGraph(x)
    graph = nx.Graph(adj_graph)

    output_band = get_bandwidth(graph, arr_pred_nodelist)
    optimal_band = get_bandwidth(graph, arr_true_nodelist)
    cuthillmckee_band = getCuthillMckeeBand(graph)

    sum_pred += output_band
    sum_true += optimal_band
    sum_rcm += cuthillmckee_band
    # print(graph)
    # print(arr_pred_nodelist)
    # print(arr_true_nodelist)
    # print(cuthillmckee_band)
    # print("bands")
    # print(output_band)
    # print(optimal_band)
    # print(cuthillmckee_band)

    df = df.append({'NN_Output': output_band, 'Optimal_band': optimal_band, 'Cuthill_Mckee_band': cuthillmckee_band}, ignore_index=True)
    # break

In [11]:
print("Averages:")
print(sum_pred / len(test_labels))
print(sum_true / len(test_labels))
print(sum_rcm / len(test_labels))

Averages:
3.4615384615384617
3.0865384615384617
3.451923076923077


In [12]:
df

,NN_Output,Optimal_band,Cuthill_Mckee_band
0,1.0,1.0,1.0
1,2.0,2.0,2.0
2,3.0,3.0,4.0
3,3.0,3.0,5.0
4,2.0,1.0,1.0
...,...,...,...
99,5.0,4.0,4.0
100,4.0,4.0,4.0
101,4.0,4.0,4.0
102,5.0,5.0,5.0


In [13]:
df.head()

,NN_Output,Optimal_band,Cuthill_Mckee_band
0,1.0,1.0,1.0
1,2.0,2.0,2.0
2,3.0,3.0,4.0
3,3.0,3.0,5.0
4,2.0,1.0,1.0


In [14]:
latex = df.to_latex(index=False)
print("Done")

Done


In [15]:
print(latex)

\begin{tabular}{rrr}
\toprule
 NN\_Output &  Optimal\_band &  Cuthill\_Mckee\_band \\
\midrule
       1.0 &           1.0 &                 1.0 \\
       2.0 &           2.0 &                 2.0 \\
       3.0 &           3.0 &                 4.0 \\
       3.0 &           3.0 &                 5.0 \\
       2.0 &           1.0 &                 1.0 \\
       2.0 &           1.0 &                 1.0 \\
       4.0 &           1.0 &                 1.0 \\
       3.0 &           2.0 &                 2.0 \\
       2.0 &           2.0 &                 2.0 \\
       2.0 &           2.0 &                 2.0 \\
       2.0 &           2.0 &                 2.0 \\
       3.0 &           3.0 &                 3.0 \\
       3.0 &           2.0 &                 3.0 \\
       3.0 &           3.0 &                 5.0 \\
       3.0 &           2.0 &                 3.0 \\
       3.0 &           3.0 &                 3.0 \\
       3.0 &           3.0 &                 4.0 \\
       3.0 &         